In [1]:
import os
import tensorflow as tf
import pandas as pd
from joblib import load

2025-10-24 14:15:53.422503: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 14:15:53.457155: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 14:15:53.651332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 14:15:54.837320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ARTIFACTS_DIR = "../artifacts"

In [3]:
(
    _X_train, _X_val, X_test,
    _y_train, _y_val, y_test,
    classes
) = load(os.path.join(ARTIFACTS_DIR, "images_tuple.joblib"))

In [4]:
model_files = [
    f for f in os.listdir(ARTIFACTS_DIR)
    if f.startswith("best_cnn_") and f.endswith(".keras")
]

results = []
for file in sorted(model_files):
    path = os.path.join(ARTIFACTS_DIR, file)
    print(f"Evaluating {file} ...")
    model = tf.keras.models.load_model(path)
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    results.append((file.replace(".keras", ""), acc, loss))

Evaluating best_cnn_baseline.keras ...
Evaluating best_cnn_batchnorm.keras ...
Evaluating best_cnn_data_augmentation.keras ...
Evaluating best_cnn_dropout.keras ...
Evaluating best_cnn_dropout_v2.keras ...
Evaluating best_cnn_pooling.keras ...


In [5]:
df = pd.DataFrame(results, columns=["Model", "Test Accuracy", "Test Loss"])
df = df.sort_values("Test Accuracy", ascending=False).reset_index(drop=True)
df

,Model,Test Accuracy,Test Loss
0,best_cnn_pooling,0.937500,0.269856
1,best_cnn_batchnorm,0.914062,0.435172
2,best_cnn_dropout_v2,0.890625,0.324246
3,best_cnn_baseline,0.867188,0.552192
4,best_cnn_dropout,0.835938,0.511460
5,best_cnn_data_augmentation,0.796875,0.715504
